In [13]:
!pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

In [14]:
import mlflow
mlflow.set_tracking_uri("http://ec2-98-89-47-151.compute-1.amazonaws.com:5000/")

In [15]:
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://mlflow-s3-bucket-25/455021568266578080', creation_time=1766405440909, experiment_id='455021568266578080', last_update_time=1766405440909, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [16]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [17]:
# Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 10000 for TF-IDF

# Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)
        
        # Print results
        print(f"✓ {model_name} - Accuracy: {accuracy:.4f}")


# Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    print("Starting XGBoost hyperparameter tuning with Optuna...")
    print("Testing 30 different parameter combinations...")
    print("=" * 60)
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30, show_progress_bar=True)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    print("\nBest parameters found:")
    print(f"  n_estimators: {best_params['n_estimators']}")
    print(f"  learning_rate: {best_params['learning_rate']:.6f}")
    print(f"  max_depth: {best_params['max_depth']}")
    print(f"  Best accuracy: {study.best_value:.4f}")
    print("=" * 60)
    
    # Create the best model with found parameters
    best_model = XGBClassifier(
        n_estimators=best_params['n_estimators'], 
        learning_rate=best_params['learning_rate'], 
        max_depth=best_params['max_depth'], 
        random_state=42
    )
    
    # Log the best model with MLflow
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)
    
    print("\n✓ Experiment completed!")
    print(f"✓ View results: http://ec2-98-89-47-151.compute-1.amazonaws.com:5000/")


# Run the experiment
run_optuna_experiment()


[I 2025-12-22 19:47:37,413] A new study created in memory with name: no-name-49cedec7-cf8b-4d33-98cb-81e1b29d5eca


Starting XGBoost hyperparameter tuning with Optuna...
Testing 30 different parameter combinations...


Best trial: 0. Best value: 0.752595:   3%|▎         | 1/30 [00:56<27:04, 56.00s/it]

[I 2025-12-22 19:48:33,500] Trial 0 finished with value: 0.75259546014429 and parameters: {'n_estimators': 170, 'learning_rate': 0.002064332445162466, 'max_depth': 10}. Best is trial 0 with value: 0.75259546014429.


Best trial: 0. Best value: 0.752595:   7%|▋         | 2/30 [01:16<16:26, 35.24s/it]

...

[I 2025-12-22 20:08:26,238] Trial 28 finished with value: 0.9026922400140771 and parameters: {'n_estimators': 199, 'learning_rate': 0.09479795071123262, 'max_depth': 10}. Best is trial 11 with value: 0.9169452753827204.


Best trial: 11. Best value: 0.916945: 100%|██████████| 30/30 [21:39<00:00, 43.32s/it]


[I 2025-12-22 20:09:17,098] Trial 29 finished with value: 0.8590533169100827 and parameters: {'n_estimators': 228, 'learning_rate': 0.029561827583511033, 'max_depth': 10}. Best is trial 11 with value: 0.9169452753827204.

Best parameters found:
  n_estimators: 299
  learning_rate: 0.096135
  max_depth: 10
  Best accuracy: 0.9169
✓ XGBoost - Accuracy: 0.9169


2025/12/22 20:10:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBoost_SMOTE_TFIDF_Trigrams at: http://ec2-98-89-47-151.compute-1.amazonaws.com:5000/#/experiments/455021568266578080/runs/c96f8eb8fa594a51ab28c7596ea77291.
2025/12/22 20:10:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-98-89-47-151.compute-1.amazonaws.com:5000/#/experiments/455021568266578080.



✓ Experiment completed!
✓ View results: http://ec2-98-89-47-151.compute-1.amazonaws.com:5000/


In [18]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [19]:
# Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)
        
        # Print results
        print(f"✓ {model_name} - Accuracy: {accuracy:.4f}")


# Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    print("Starting XGBoost hyperparameter tuning with Optuna...")
    print("Testing 30 different parameter combinations...")
    print("=" * 60)
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30, show_progress_bar=True)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    print("\nBest parameters found:")
    print(f"  n_estimators: {best_params['n_estimators']}")
    print(f"  learning_rate: {best_params['learning_rate']:.6f}")
    print(f"  max_depth: {best_params['max_depth']}")

    print(f"  Best accuracy: {study.best_value:.4f}")

    print("=" * 60)
    run_optuna_experiment()

    # Run the experiment for XGBoost

    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    print(f"✓ View results: http://ec2-98-89-47-151.compute-1.amazonaws.com:5000/")

    # Log the best model with MLflow    print("\n✓ Experiment completed!")

    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)    